# Marking and saving array points and images in a single file.

In [60]:
!wget -O images/face.jpg https://upload.wikimedia.org/wikipedia/commons/3/33/Arnold_Schwarzenegger_edit%28ws%29.jpg

--2017-07-27 08:32:09--  https://upload.wikimedia.org/wikipedia/commons/3/33/Arnold_Schwarzenegger_edit%28ws%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89175 (87K) [image/jpeg]
Saving to: ‘images/face-to-crop.jpg’

images/face-to-crop 100%[===================>]  87.08K   184KB/s    in 0.5s    

2017-07-27 08:32:10 (184 KB/s) - ‘images/face-to-crop.jpg’ saved [89175/89175]



In [59]:
# Notice the mpld3 module - it will allow youtoload animage and use to mouse to see pixel locations
# handy tool to find points for a fast crop
import matplotlib
import numpy as np
import mpld3
import matplotlib.pyplot as plt
from PIL import Image
from mpld3 import plugins
%matplotlib inline
fig, ax = plt.subplots()
im = np.array(Image.open('images/face.png'))
plt.imshow(im)
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

In [9]:
# This is tool to mark the points and record them. The display is formated specially to useina cut and paste
# where the image arrays points are required.
import Tkinter
from PIL import Image, ImageTk
from sys import argv

window = Tkinter.Tk(className="Array Points")

image = Image.open("images/face.jpg")
canvas = Tkinter.Canvas(window, width=image.size[0], height=image.size[1])
canvas.pack()
image_tk = ImageTk.PhotoImage(image)
canvas.create_image(image.size[0]//2, image.size[1]//2, image=image_tk)

def callback(event):
    #This is formated so the array results below may be cut and pasted.
    #Don'tforget toremove the final trailing comma the pasted area should start and end with [ ]
    print "[",event.x,",",event.y,"],"
 
canvas.bind("<Button-1>", callback)
Tkinter.mainloop()

[ 287 , 347 ],
[ 408 , 481 ],


In [61]:
from PIL import Image
img = Image.open("images/face.jpg")
# this is not required to be exact I round themoff to round, near numbers 
image0 = img.crop((270,350, 408, 500))
image0.save("images/nose.jpg")

In [62]:
!showme images/nose.jpg

In [21]:
# this will will provide a finer tune ' croping array ' for seamless cloning
import Tkinter
from PIL import Image, ImageTk
from sys import argv

window = Tkinter.Tk(className="Array Points")

image = Image.open("images/nose.jpg")
canvas = Tkinter.Canvas(window, width=image.size[0], height=image.size[1])
canvas.pack()
image_tk = ImageTk.PhotoImage(image)
canvas.create_image(image.size[0]//2, image.size[1]//2, image=image_tk)

def callback(event):
    #print "[",event.x,"],["+event.y,"]"
    print "[",event.x,",",event.y,"],"
 
canvas.bind("<Button-1>", callback)
Tkinter.mainloop()

[ 33 , 21 ],
[ 3 , 111 ],
[ 11 , 135 ],
[ 69 , 144 ],
[ 122 , 137 ],
[ 129 , 92 ],
[ 75 , 14 ],
[ 54 , 12 ],
[ 65 , 12 ],


# Saving multiple arrays in an npz file by example:

In [72]:
# Saving multiple arrays in an npz file example:
# Create two arrays
x = np.arange(10)
y = np.sin(x)

#Using savez with *args, the arrays are saved with default names " ['arr_1', 'arr_0'] "
np.savez("numpy-filters/temp1", x, y)

In [73]:
npz = np.load("numpy-filters/temp1.npz")
npz.files

['arr_1', 'arr_0']

In [74]:
# Retrieve the first array for use
npz['arr_0']

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [76]:
# Retrieve the second array for use
npz['arr_1']
# Assigning a variable to the array
sin = npz['arr_1']
print sin

[ 0.          0.84147098  0.90929743  0.14112001 -0.7568025  -0.95892427
 -0.2794155   0.6569866   0.98935825  0.41211849]


### Existing variable names may be used. The input form must be var = var - example: oranges=oranges

In [84]:
oranges = np.arange(10)
apples = np.sin(x)
np.savez("numpy-filters/example2", oranges=oranges, apples=apples)

In [85]:
npzfile = np.load("numpy-filters/example2.npz")
npzfile.files

['apples', 'oranges']

In [87]:
npzfile['apples'] , npzfile['oranges'] 

(array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
        -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

# Now, to save some numpy arrays and use them

In [91]:
import cv2
import numpy as np 

# Read face image ( dst destination of the nose src )
dst = cv2.imread("images/face.jpg")
# Read nose image ( src of the seamlesClone image )
src = cv2.imread("images/nose.jpg")


# Create a rough mask around the nose
src_mask = np.zeros(src.shape, src.dtype)

# notice the array above is cut and pasted here
poly = np.array([[ 33 , 21 ],
[ 3 , 111 ],
[ 11 , 135 ],
[ 69 , 144 ],
[ 122 , 137 ],
[ 129 , 92 ],
[ 75 , 14 ],
[ 65 , 12 ]], np.int32)

# This will save BOTH the nose image and the poly array in a single file src-mask001.npz
# notice below,  I am using src=src, poly=poly . 
#That will give me arrays npz['src'] and npz['poly']

np.savez('numpy-filters/nose_001', src=src, poly=poly) 

#place the nose image poly array in the 'rough' source mask
cv2.fillPoly(src_mask, [poly], (255, 255, 255))

# This is where the CENTER of the airplane will be placed
center = (340,255)

# Clone seamlessly.
output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)

# Write result
cv2.imwrite("images/face_clone.jpg", output);
!showme images/face_clone.jpg

# Use our newly created 
# ' numpy-filters/src-mask001.npz '  on another image

In [42]:
# Get an image from public domain wiki
!wget -O images/woman.jpg https://upload.wikimedia.org/wikipedia/commons/1/1b/Young_Woman_Thinking.jpg

--2017-07-27 07:42:45--  https://upload.wikimedia.org/wikipedia/commons/1/1b/Young_Woman_Thinking.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101797 (99K) [image/jpeg]
Saving to: ‘images/woman.jpg’

images/woman.jpg    100%[===================>]  99.41K   194KB/s    in 0.5s    

2017-07-27 07:42:47 (194 KB/s) - ‘images/woman.jpg’ saved [101797/101797]



In [43]:
#get points for cropping image
import Tkinter
from PIL import Image, ImageTk
from sys import argv

window = Tkinter.Tk(className="Array Points")

image = Image.open("images/woman.jpg")
canvas = Tkinter.Canvas(window, width=image.size[0], height=image.size[1])
canvas.pack()
image_tk = ImageTk.PhotoImage(image)
canvas.create_image(image.size[0]//2, image.size[1]//2, image=image_tk)

def callback(event):
    #print "[",event.x,"],["+event.y,"]"
    print "[",event.x,",",event.y,"],"
 
canvas.bind("<Button-1>", callback)
Tkinter.mainloop()

[ 372 , 76 ],
[ 959 , 590 ],


In [48]:
# Crop the Downloaded, images/woman.jpg, image.
from PIL import Image
img = Image.open("images/woman.jpg")

image0 = img.crop((350,70, 950, 600))
image0.save("images/Crop_woman.jpg")

In [49]:
# Look at the image
!showme images/Crop_woman.jpg

In [93]:
# View the files/arrays in the " nose_001.npz "
npz = np.load("numpy-filters/nose_001.npz")
npz.files

['src', 'poly']

In [92]:
import cv2
import numpy as np 

# Read woman image
dst = cv2.imread("images/Crop_woman.jpg")

#load the nose image AND the numpy array around it
npz = np.load("numpy-filters/nose_001.npz")


# The src_mask requires shape from the original nose image
# be sure define this variable before creating the src_mask 
src=npz['src']

# cv2.fillPoly requires placing the poly array in the src_mask
poly=npz['poly']

#rough shape a mask
src_mask = np.zeros(src.shape, src.dtype)

#fill that rough mask with the poly array of the select areas of the nose image
# nose_001a.npz contains the nose image and the poly array
cv2.fillPoly(src_mask, [poly], (255, 255, 255))

# This is where the CENTER where the nose image will be placed
center = (265,210)

# Clone seamlessly.
output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)

# Write result
cv2.imwrite("images/face_clonez.jpg", output);
!showme images/face_clonez.jpg

In [98]:
npz = np.load("numpy-filters/nose_001.npz")
src=npz['src']
poly=npz['poly']
print "POLY: ",poly , "\n \n SRC: ",src

POLY:  [[ 33  21]
 [  3 111]
 [ 11 135]
 [ 69 144]
 [122 137]
 [129  92]
 [ 75  14]
 [ 65  12]] 
 
 SRC:  [[[149 208 255]
  [150 210 255]
  [152 211 255]
  ..., 
  [146 202 251]
  [154 209 255]
  [154 209 255]]

 [[147 206 255]
  [146 205 254]
  [147 205 254]
  ..., 
  [142 197 248]
  [148 201 252]
  [146 201 252]]

 [[150 206 255]
  [146 203 255]
  [142 199 251]
  ..., 
  [135 187 240]
  [140 190 243]
  [138 190 243]]

 ..., 
 [[ 97 150 241]
  [ 97 150 241]
  [ 97 150 241]
  ..., 
  [ 84 131 222]
  [ 84 131 222]
  [ 84 131 222]]

 [[ 97 150 241]
  [ 97 150 241]
  [ 97 150 241]
  ..., 
  [ 84 132 220]
  [ 86 134 222]
  [ 86 134 222]]

 [[ 97 150 241]
  [ 97 150 241]
  [ 97 150 241]
  ..., 
  [ 84 132 220]
  [ 87 135 223]
  [ 87 135 223]]]
